In [3]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import io
import requests
import re
from bs4 import BeautifulSoup
from functools import partial, reduce
from download_data import get_url, clean_dataframe, get_csv
from transform_data import *
import os
import glob
import boto3
from to_csv_on_s3 import to_csv_on_s3

## 1-2. パラメーターの設定
# 元データをダウンロードするためのフォルダ
# reading s3 bucket
client = boto3.client('s3')

dirname = os.getcwd()
# FOLDER_RAW = os.path.join(dirname, '元データ/')
BUCKET_NAME = 'jpx-future-bucket'
FOLDER_RAW = "raw_data/"

# 完成したデータを保存するためのフォルダ
# FOLDER_FINAL = os.path.join(dirname, '完成データ/')
# FOLDER_FINAL = 's3://jpx-future-bucket/完成データ/'
FOLDER_FINAL = "final_data/"

# 企業のグループ分けに使うExcelファイル
PATH_GROUP = os.path.join(dirname, '企業名グループ.xlsx')

# TOPIXに掛ける係数（225合計と合わせて総合計を計算する際に使う）
TOPIX_MULTI = 1.0

ModuleNotFoundError: No module named 'download_data'

In [2]:
days = 0
print("days", days)
print("TOPIX_MULTI", TOPIX_MULTI)

day = dt.today() - timedelta(days=days)
colnames = ['institutions_sell_code', 'institutions_sell', 
        'institutions_sell_eng', 'volume_sell', 'institutions_buy_code', 
        'institutions_buy', 'institutions_buy_eng', 'volume_buy']

print("2. データのダウンロード")

### 2-2. 日中立会取引データのダウンロード
url_wholeday = get_url(datatype=3, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日の日中立会取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_wholeday).ok:
    s=requests.get(url_wholeday).content
    df_wholeday = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "日中立会取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_wholeday.to_csv(filename)
    to_csv_on_s3(df_wholeday, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日の日中立会取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

### 2-3. 日中JNET取引データのダウンロード
url_wholeday_JNET = get_url(datatype=4, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日の日中JNET取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_wholeday_JNET).ok:
    s=requests.get(url_wholeday_JNET).content
    df_wholeday_JNET = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "日中JNET取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_wholeday_JNET.to_csv(filename)
    to_csv_on_s3(df_wholeday_JNET, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日の日中JNET取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

### 2-4. ナイト立会取引データのダウンロード
url_night = get_url(datatype=1, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日のナイト立会取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_night).ok:
    s=requests.get(url_night).content
    df_night = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "ナイト立会取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_night.to_csv(filename)
    to_csv_on_s3(df_night, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日のナイト立会取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

### 2-5. ナイトJNET取引データのダウンロード
url_night_JNET = get_url(datatype=2, day=day)
if url_wholeday == "No data":
    # 休日などでファイルがない場合はエラーメッセージを返します
    text = "%s年%s月%s日のナイトJNET取引データが見つかりません"%(day.year, day.month, day.day)
    print(text)
elif url_wholeday == "No page":
    text = "メインページが見つかりません。https://www.jpx.co.jp/markets/derivatives/participant-volume/index.html　を確認してください"
    print(text)
elif requests.get(url_night_JNET).ok:
    s=requests.get(url_night_JNET).content
    df_night_JNET = clean_dataframe(get_csv(s, colnames = colnames), day = day)
    filename = FOLDER_RAW + "ナイトJNET取引" + str(day.year) +"-"+ str(day.month) +"-"+ str(day.day) + ".csv"
    # df_night_JNET.to_csv(filename)
    to_csv_on_s3(df_night_JNET, bucketName=BUCKET_NAME, fileName=filename)
    text = "%s年%s月%s日のナイトJNET取引データをダウンロードしました"%(day.year, day.month, day.day)
    print(text)
else:
    text = "何らかの理由でデータが見つかりません"
    print(text)

days 0


NameError: name 'TOPIX_MULTI' is not defined